In [20]:
import numpy as np
import pandas as pd
import re
import nltk
import string
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from nltk import ngrams
from collections import Counter

## Define Paths

These paths assume that the code is running from the location where this notebook is saved. If you want to run from a different location change the paths accordingly

In [7]:
data_path = "Data/"
nltk_path = "NLTK/"

## Load the Data

In [8]:
train = pd.read_csv(data_path+'train.csv')
test = pd.read_csv(data_path+'test.csv')

#Mix the data so that real and fake accounts are not clustered
train = train.reindex(np.random.permutation(train.index)) 

In [9]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because",
                       "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not",
                       "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would",
                       "he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", 
                       "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have",
                       "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                       "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am",
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have",
                       "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us",
                       "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not",
                       "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                       "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have",
                       "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have",
                       "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                       "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is",
                       "they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have",
                       "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would",
                       "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have",
                       "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is",
                       "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                       "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                       "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                       "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                       "you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have",
                       "you're": "you are", "you've": "you have" }

In [10]:
def split_textnum(text):
    '''
    To seperate numbers from the words.
    
    Input - Word
    
    Returns - Number seperated list of items.
    '''
    match = re.match(r"([a-z]+)([0-9]+)", text, re.I)
    if match:
        items = " ".join(list(match.groups()))
    else:
        match = re.match(r"([0-9]+)([a-z]+)", text, re.I)
        if match:
            items = " ".join(list(match.groups()))
        else:
            return text
    return (items)

In [11]:
# Let's remove special charecters and unwanted datas. This is totally manual task, kind of real pain of NLP data cleaning.
def clean_text(text): 
            
    # Special characters
    text = re.sub(r"%20", " ", text)
    #text = text.replace(r".", " ")
    text = text.replace(r"@", " ")
    text = text.replace(r"#", " ")
    #text = text.replace(r":", " ")
    text = text.replace(r"'", " ")
    text = text.replace(r"\x89û_", " ")
    text = text.replace(r"??????", " ")
    text = text.replace(r"\x89ûò", " ")
    text = text.replace(r"16yr", "16 year")
    text = text.replace(r"re\x89û_", " ")
    
    text = text.replace(r"mh370", " ")
    text = text.replace(r"prebreak", "pre break")
    text = re.sub(r"\x89û", " ", text)
    text = re.sub(r"re\x89û", "re ", text)
    text = text.replace(r"nowplaying", "now playing")
    text = re.sub(r"\x89ûª", "'", text)
    text = re.sub(r"\x89û", " ", text)
    text = re.sub(r"\x89ûò", " ", text)
    
    
    text = re.sub(r"\x89Û_", "", text)
    text = re.sub(r"\x89ÛÒ", "", text)
    text = re.sub(r"\x89ÛÓ", "", text)
    text = re.sub(r"\x89ÛÏWhen", "When", text)
    text = re.sub(r"\x89ÛÏ", "", text)
    text = re.sub(r"China\x89Ûªs", "China's", text)
    text = re.sub(r"let\x89Ûªs", "let's", text)
    text = re.sub(r"\x89Û÷", "", text)
    text = re.sub(r"\x89Ûª", "", text)
    text = re.sub(r"\x89Û\x9d", "", text)
    text = re.sub(r"å_", "", text)
    text = re.sub(r"\x89Û¢", "", text)
    text = re.sub(r"\x89Û¢åÊ", "", text)
    text = re.sub(r"fromåÊwounds", "from wounds", text)
    text = re.sub(r"åÊ", "", text)
    text = re.sub(r"åÈ", "", text)
    text = re.sub(r"JapÌ_n", "Japan", text)    
    text = re.sub(r"Ì©", "e", text)
    text = re.sub(r"å¨", "", text)
    text = re.sub(r"SuruÌ¤", "Suruc", text)
    text = re.sub(r"åÇ", "", text)
    text = re.sub(r"å£3million", "3 million", text)
    text = re.sub(r"åÀ", "", text)
    
    text = re.sub(r'http\S+', ' ', text)
    text = re.sub(r"ªs", " ", text)
    text = re.sub(r"ª", " ", text)
    text = re.sub(r"\x9d", " ", text)
    text = re.sub(r"ò", " ", text)
    text = re.sub(r"ªt", " ", text)
    text = re.sub(r"ó", " ", text)
    text = text.replace(r"11yearold", "11 year old")
    text = re.sub(r"typhoondevastated", "typhoon devastated", text)
    text = re.sub(r"bestnaijamade", "best nijamade", text)
    text = re.sub(r"gbbo", "The Great British Bake Off", text)
    text = re.sub(r"ï", "", text)
    text = re.sub(r"ïwhen", "when", text)
    text = re.sub(r"selfimage", "self image", text)
    text = re.sub(r"20150805", "2015 08 05", text)
    text = re.sub(r"20150806", "2015 08 06", text)
    text = re.sub(r"subreddits", "website for weird public sentiment", text)
    text = re.sub(r"disea", "chinese famous electronic company", text)
    text = re.sub(r"lmao", "funny", text)
    text = text.replace(r"companyse", "company")
    
    text = text.replace(r"worldnews", "world news")
    text = text.replace(r"animalrescue", "animal rescue")
    text = text.replace(r"freakiest", "freak")
    
    text = text.replace(r"irandeal", "iran deal")
    text = text.replace(r"directioners", "mentor")
    text = text.replace(r"justinbieber", "justin bieber")
    text = text.replace(r"okwx", "okay")
    text = text.replace(r"trapmusic", "trap music")
    text = text.replace(r"djicemoon", "music ice moon")
    text = text.replace(r"icemoon", "ice moon")
    text = text.replace(r"mtvhottest", "tv hottest")
    text = text.replace(r"rì©union", "reunion")
    text = text.replace(r"abcnews", "abc news")
    text = text.replace(r"tubestrike", "tube strike")
    text = text.replace(r"prophetmuhammad", "prophet muhammad muslim dharma")
    text = text.replace(r"chicagoarea", "chicago area")
    text = text.replace(r"yearold", "year old")
    text = text.replace(r"meatloving", "meat love")
    text = text.replace(r"standuser", "standard user")
    text = text.replace(r"pantherattack", "panther attack")
    text = text.replace(r"youngheroesid", "young hearos id")
    text = text.replace(r"idk", "i do not know")
    text = text.replace(r"usagov", "united state of america government")
    text = text.replace(r"injuryi", "injury")
    text = text.replace(r"summerfate", "summer fate")
    text = text.replace(r"kerricktrial", "kerrick trial")
    text = text.replace(r"viralspell", "viral spell")
    text = text.replace(r"collisionno", "collision")
    text = text.replace(r"socialnews", "social news")
    text = text.replace(r"nasahurricane", "nasa hurricane")
    text = text.replace(r"strategicpatience", "strategic patience")
    text = text.replace(r"explosionproof", "explosion proof")
    text = text.replace(r"selfies", "photo")
    text = text.replace(r"selfie", "photo")
    text = text.replace(r"worstsummerjob", "worst summer job")
    text = text.replace(r"realdonaldtrump", "real america president")
    text = text.replace(r"omfg", "oh my god")
    text = text.replace(r"japìn", "japan")
    text = text.replace(r"breakingnews", "breaking news")
    
    text = " ".join([split_textnum(word) for word in text.split(" ")])
    
    text = "".join([c if c not in string.punctuation else "" for c in text])
    text = ''.join(c for c in text if not c.isdigit())
    text = text.replace(r"÷", "")
    
    text = re.sub(' +', ' ', text)
    # text = text.encode('utf-8')
    return text

In [13]:
train['text_processed'] = train['text'].apply(lambda x : " ".join([contraction_mapping[word].lower() 
                    if word in contraction_mapping.keys() else word.lower() for word in x.split(" ")]))
test['text_processed'] = test['text'].apply(lambda x : " ".join([contraction_mapping[word].lower() 
                    if word in contraction_mapping.keys() else word.lower() for word in x.split(" ")]))
train['text_processed'] = train['text_processed'].apply(lambda x : clean_text(x))
test['text_processed'] = test['text_processed'].apply(lambda x : clean_text(x))

In [14]:
nltk.data.path.append(nltk_path + "nltk_data/")
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
train['text_processed'] = train['text_processed'].apply(lambda x : "".join([lemmatizer.lemmatize(word) 
                                                                            for word in x]))
test['text_processed'] = test['text_processed'].apply(lambda x : "".join([lemmatizer.lemmatize(word) 
                                                                          for word in x]))

## Bag of Words

In [15]:
def cv(data):
    count_vectorizer = CountVectorizer()
    emb = count_vectorizer.fit_transform(data)
    return emb, count_vectorizer

## TF-IDF

In [16]:
def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer()
    train = tfidf_vectorizer.fit_transform(data)
    return train, tfidf_vectorizer

## Train-Validation split

In [18]:
list_corpus = train["text_processed"].tolist()
list_labels = train["target"].tolist()

X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, test_size=0.2)

In [21]:
#Bag of Words embeddings
X_train_counts, count_vectorizer = cv(X_train)
X_test_counts = count_vectorizer.transform(X_test)

#TF-IDF embeddings
X_train_tfidf, tfidf_vectorizer = tfidf(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)